<a href="https://colab.research.google.com/github/aravind-gk/Deep-Learning-for-NLP-Assignments/blob/main/Assignment-5/Transformer_BBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import csv
import string
import re
from tensorflow.keras import models,layers,Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet

import nltk
nltk.download('stopwords')

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive


# Loading the data

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/DL-NLP-A4/train.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/Datasets/DL-NLP-A4/test.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Datasets/DL-NLP-A4/Assignment4_TestLabels.csv")
df_test = df_test[['Text', 'Label - (business, tech, politics, sport, entertainment)']]
df_test.columns = ['Text', 'Category']

In [3]:
# swap labels with integers
df.Category=df.Category.replace("business",0)
df.Category=df.Category.replace("tech",1)
df.Category=df.Category.replace("entertainment",2)
df.Category=df.Category.replace("sport",3)
df.Category=df.Category.replace("politics",4)

df_test.Category = df_test.Category.replace("business",0)
df_test.Category = df_test.Category.replace("tech",1)
df_test.Category = df_test.Category.replace("entertainment",2)
df_test.Category = df_test.Category.replace("sport",3)
df_test.Category = df_test.Category.replace("politics",4)

In [18]:
df.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,0
1,german business confidence slides german busin...,0
2,bbc poll indicates economic gloom citizens in ...,0
3,lifestyle governs mobile choice faster bett...,1
4,enron bosses in $168m payout eighteen former e...,0


In [21]:
df_test.head()

,Text,Category
0,qpr keeper day heads for preston queens park r...,3
1,software watching while you work software that...,1
2,d arcy injury adds to ireland woe gordon d arc...,3
3,india s reliance family feud heats up the ongo...,0
4,boro suffer morrison injury blow middlesbrough...,3


In [20]:
print(df.shape)
print(df_test.shape)

(1490, 2)
(735, 3)


# Prepare the data

In [4]:
# Functions for cleaning the data
def is_special(text):
    rem = ''
    for i in text: 
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
            rem=rem+i
            rem = rem + ' '
    return rem

def rem_extra(text):
    rem=re.sub(' +', ' ',text)
    return str(rem).strip()

def get_max_length(df):
    max_length = 0
    length=[]
    for row in df['Text']:
        length.append(len(row.split(" ")))
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length,length
    
df.Text=df.Text.apply(is_special)
df.Text=df.Text.apply(rem_extra)

df_test.Text=df_test.Text.apply(is_special)
df_test.Text=df_test.Text.apply(rem_extra)

In [5]:
token=keras.preprocessing.text.Tokenizer(filters='"#$*+/:;<=>@[\\]^_{|}~\t\n')
token.fit_on_texts(df.Text)
size_of_vocabulary=len(token.word_index)+1
size_of_vocabulary

24792

In [6]:
def prepare_text(text, token, max_len):
    text_seqs = token.texts_to_sequences(text)
    return tf.keras.preprocessing.sequence.pad_sequences(text_seqs, maxlen = max_len)

max_len=1000
data_train = prepare_text(df.Text, token, max_len)
data_test = prepare_text(df_test.Text, token, max_len)

In [7]:
X_train = data_train
X_test = data_test
y_train = df.Category.values
y_test = df_test.Category.values

# Load word embeddings

In [8]:
def load_emb(addrs,size_of_vocabulary,tk):
    embeddings_index = dict()
    f = open(addrs)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    
    embedding_matrix = np.zeros((size_of_vocabulary, 300))
    c=0
    for word, i in tk.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            c+=1
        else:
            pass
    print('No. of out of vocab word in train set= %s'%(size_of_vocabulary-c))
    return embedding_matrix

def w2v():
  from gensim import models
  w2v = models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Embeddings/GoogleNews-vectors-negative300.bin', binary=True)
  embd_w2v = np.zeros((size_of_vocabulary, 300))
  c=0
  for word, i in token.word_index.items():
      try:
        embd_w2v[i] =w2v[word]   
      except:
        c+=1
        
  print('No. of out of vocab word in train set= %s'%(c))
  return embd_w2v

In [9]:
# fasttext = load_emb("/content/drive/MyDrive/Embeddings/wiki-news-300d-1M-subword.vec",size_of_vocabulary,token)
glove = load_emb("/content/drive/MyDrive/Embeddings/glove.6B.300d.txt",size_of_vocabulary,token)
# word2vec = w2v()

Loaded 400000 word vectors.
No. of out of vocab word in train set= 1053


# Define the transformer model

In [10]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


# Without positional embeddings

In [11]:
# without positional embeddings
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, weight_matrix, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, weights = [weight_matrix], trainable = False)
        # self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        # maxlen = tf.shape(x)[-1]
        # positions = tf.range(start=0, limit=maxlen, delta=1)
        # positions = self.pos_emb(positions)
        x = self.token_emb(x)
        # return x + positions
        return x

In [12]:
embed_dim = 300  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
vocab_size = size_of_vocabulary

inputs = layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, glove, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(5, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [13]:
# using glove, without position embeddings
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test)
)

Epoch 1/10
24/24 [==============================] - 21s 705ms/step - loss: 1.5640 - accuracy: 0.3148 - val_loss: 1.2741 - val_accuracy: 0.4898
Epoch 2/10
24/24 [==============================] - 17s 695ms/step - loss: 1.1845 - accuracy: 0.4638 - val_loss: 0.9786 - val_accuracy: 0.5551
Epoch 3/10
24/24 [==============================] - 17s 700ms/step - loss: 0.9285 - accuracy: 0.5711 - val_loss: 0.7451 - val_accuracy: 0.6422
Epoch 4/10
24/24 [==============================] - 17s 704ms/step - loss: 0.6532 - accuracy: 0.7450 - val_loss: 0.5008 - val_accuracy: 0.7837
Epoch 5/10
24/24 [==============================] - 17s 706ms/step - loss: 0.2916 - accuracy: 0.9027 - val_loss: 0.2942 - val_accuracy: 0.9524
Epoch 6/10
24/24 [==============================] - 17s 706ms/step - loss: 0.1505 - accuracy: 0.9604 - val_loss: 0.2106 - val_accuracy: 0.9551
Epoch 7/10
24/24 [==============================] - 17s 706ms/step - loss: 0.0795 - accuracy: 0.9805 - val_loss: 0.1895 - val_accuracy: 0.9605

In [14]:
y_pred = model.predict(data_test)
print(classification_report(y_test, np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94       176
           1       0.94      0.98      0.96       135
           2       0.96      0.98      0.97       113
           3       1.00      0.97      0.98       168
           4       0.96      0.98      0.97       143

    accuracy                           0.96       735
   macro avg       0.96      0.97      0.96       735
weighted avg       0.97      0.96      0.96       735



In [17]:
model.save(filepath = '/content/drive/MyDrive/Models/DL-NLP-A5/transformer-BBC-withoutPE')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Models/DL-NLP-A5/transformer-BBC-withoutPE/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Models/DL-NLP-A5/transformer-BBC-withoutPE/assets


# With positional embeddings

In [18]:
# with positional embeddings
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, weight_matrix, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, weights = [weight_matrix], trainable = False)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
        return x

In [21]:
embed_dim = 300  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
vocab_size = size_of_vocabulary

inputs = layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, glove, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(5, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [22]:
# using glove, without position embeddings
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test)
)

Epoch 1/10
24/24 [==============================] - 20s 769ms/step - loss: 1.5974 - accuracy: 0.3020 - val_loss: 1.3878 - val_accuracy: 0.4980
Epoch 2/10
24/24 [==============================] - 18s 771ms/step - loss: 1.0419 - accuracy: 0.6289 - val_loss: 0.6959 - val_accuracy: 0.8000
Epoch 3/10
24/24 [==============================] - 18s 773ms/step - loss: 0.5202 - accuracy: 0.7490 - val_loss: 0.3611 - val_accuracy: 0.9415
Epoch 4/10
24/24 [==============================] - 19s 777ms/step - loss: 0.2080 - accuracy: 0.9409 - val_loss: 0.2133 - val_accuracy: 0.9510
Epoch 5/10
24/24 [==============================] - 19s 779ms/step - loss: 0.0834 - accuracy: 0.9752 - val_loss: 0.2183 - val_accuracy: 0.9551
Epoch 6/10
24/24 [==============================] - 19s 781ms/step - loss: 0.0664 - accuracy: 0.9758 - val_loss: 0.2186 - val_accuracy: 0.9605
Epoch 7/10
24/24 [==============================] - 19s 779ms/step - loss: 0.0349 - accuracy: 0.9913 - val_loss: 0.2135 - val_accuracy: 0.9592

In [23]:
y_pred = model.predict(data_test)
print(classification_report(y_test, np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94       176
           1       0.94      0.99      0.96       135
           2       0.96      0.95      0.95       113
           3       1.00      0.98      0.99       168
           4       0.97      0.95      0.96       143

    accuracy                           0.96       735
   macro avg       0.96      0.96      0.96       735
weighted avg       0.96      0.96      0.96       735



In [24]:
model.save(filepath = '/content/drive/MyDrive/Models/DL-NLP-A5/transformer-BBC-withPE')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Models/DL-NLP-A5/transformer-BBC-withPE/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Models/DL-NLP-A5/transformer-BBC-withPE/assets
